In [ ]:
from docs_jpmml_org import fetch_bank_marketing

dataset = fetch_bank_marketing()
#print(dataset)

X, y = dataset.getData()

y = y["y"]

categorical_cols = dataset.getCategoricalColumns()
continuous_cols = dataset.getContinuousColumns()

In [ ]:
from pandas import CategoricalDtype
from xgboost import DMatrix

import xgboost

X = X[continuous_cols + categorical_cols]

X[continuous_cols] = X[continuous_cols].astype("Float32")
X[categorical_cols] = X[categorical_cols].astype("category")

y = y.astype(CategoricalDtype(categories = ["no", "yes"]))

feature_types = ["q"] * len(continuous_cols) + ["c"] * len(categorical_cols)

dmat = DMatrix(data = X, label = y.cat.codes, feature_types = feature_types, enable_categorical = True)

booster = xgboost.train(params = {"objective" : "binary:logistic"}, dtrain = dmat)
#print(booster)

booster.save_model("Booster.json")

In [ ]:
from sklearn2pmml import sklearn2pmml
from sklearn2pmml.xgboost import make_feature_map

fmap = make_feature_map(X, category_to_indicator = True)
fmap.save("Booster.fmap.tsv")

booster.fmap = fmap

sklearn2pmml(booster, "Booster.pmml")